### 検証内容

以下のストーリーを基に検証を行う：
https://www.pivotaltracker.com/n/projects/1879711/stories/154470533

* 今回検証したrocchioの動き自体が本当にその通りなのか今一度確認する
   * https://www.pivotaltracker.com/n/projects/1879711/stories/154470533
* 本当にrocchioがダメな場合別のアルゴリズムを探したい

### アルゴリズム

Rocchio のアルゴリズムは、ユーザークエリ $\boldsymbol{q}$ に対して、関連する書類の特徴ベクトルの集合 $D_{\mathrm{r}}$ と関連しない書類の特徴ベクトルの集合 $D_{\mathrm{nr}}$ を定義し、次式の更新式によってクエリを修正し、検索を行うアルゴリズムである。

\begin{equation}
\boldsymbol{q^{*}} = \alpha \boldsymbol{q} + \frac{\beta}{|D_{\mathrm{r}}|} \sum_{\boldsymbol{d} \in D_{\mathrm{r}}} \boldsymbol{d} - \frac{\gamma}{|D_{\mathrm{nr}}|} \sum_{\boldsymbol{d} \in D_{\mathrm{nr}}} \boldsymbol{d}
\end{equation}

但、$\alpha, \beta, \gamma$ はそれぞれ実数である。
従って、関連するドキュメントと関連しないドキュメントの重心をそれぞれ求め、クエリをその方向に修正するアルゴリズムとなる。

In [48]:
#########################################################
#
#   検証用のスクリプト：結果は markdown形式で書き直すので、読まなくても良い
#
#########################################################


import numpy as np


#
# ngram tokenizer
#
def ngram(text, n):
    return [text[i:i+n] for i in range(len(text))]


#
# コーパスを生成する
#
def prepare_corpus(docs):
    corpus = {}
    for d in docs:
        grams = ngram(d, 2)
        for g in grams:
            if g in corpus:
                corpus[g]['n'] += 1
            else:
                corpus[g] = {'n': 1, 'id': len(corpus)}
    return corpus


#
# 文章とコーパスから特徴ベクトルを生成する
#
def convert_doc_to_vector(doc, corpus):
    grams = ngram(doc, 2)
    dim = len(corpus)
    fv = np.zeros(dim)
    for g in grams:
        if g in corpus:
            id = corpus[g]['id']
            fv[id] = 1.0
    return fv


#
# コサイン類似度
#
def cos(x, y):
    return np.inner(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))


#
# 近傍探索
#
def nearest(query, docs):
    v_query = convert_doc_to_vector(query, corpus)
    return _nearest(v_query, docs)


#
# クエリがベクトルの場合の近傍探索
#
def _nearest(v_query, docs):
    corpus = prepare_corpus(docs)
    
    res = []
    for doc in docs:
        v_doc = convert_doc_to_vector(doc, corpus)
        res.append([doc, cos(v_query, v_doc)])
    
    return sorted(res, key=lambda x: -x[1])


#
# Rocchioアルゴリズム
#
def Rocchio(q, d_r=[], d_nr=[], alpha=0.6, beta=0.3, gamma=0.1):
    a = alpha * q
    b = beta * np.sum(d_r, axis=0) / len(d_r)
    c =  gamma * np.sum(d_nr, axis=0) / len(d_nr)
    return a + b - c


#
# 対象となる文章
#
docs = [
    "明日の会議室はとれましたか？",
    "明日の打合せの予定はどうなっていますか？",
    "僕の iPhone は床に叩きつけて壊れました。",
    "あなたの iPad は、町内会の福引の景品でもらったものですね？",
    "虚数単位 i は、自乗して -1 になる数と定義したものです。",
]

corpus = prepare_corpus(docs)


#
# ユーザークエリ
#
q1 = "明日のMTGってどうなった？"
q2 = "i の2乗っていくつになるか知ってる？"


#
# それぞれの文書の関係を定義する
#
relation = {
    '明日のMTGってどうなった？': {
        'pos': [0, 1], 'neg': [2,3,4],
    },
    'i の2乗っていくつになるか知ってる？': {
        'pos': [4], 'neg': [0,1,2,3],
    },
}

print("===== 何もフィードバックしない状態で検索 =====")
print("クエリ：{0}".format(q1))
print("=================================")
for r in nearest(q1, docs):
    print("一致度：{0} | {1}".format(r[1], r[0]))

print("")
print("クエリに Rocchioアルゴリズムによるフィードバックを与えてみる。")
print("")
d_pos = [docs[i] for i in relation[q1]['pos']]
d_neg = [docs[i] for i in relation[q1]['neg']]
print("pos: {}".format(d_pos))
print("neg: {}".format(d_neg))

v_pos = [convert_doc_to_vector(x, corpus) for x in d_pos]
v_neg = [convert_doc_to_vector(x, corpus) for x in d_neg]

v_q1 = convert_doc_to_vector(q1, corpus)
q = Rocchio(v_q1, v_pos, v_neg)
print("")
print("===== Rocchioアルゴリズムを適用した状態で検索 =====")
print("クエリ：{0}".format(q1))
print("=================================")
for r in _nearest(q, docs):
    print("一致度：{0} | {1}".format(r[1], r[0]))

print("")
print("iPad の件の文章の一致度が下がってていい感じ")
print("続いてもう一つのクエリについて")
print("")


print("===== 何もフィードバックしない状態で検索 =====")
print("クエリ：{0}".format(q2))
print("=================================")
for r in nearest(q2, docs):
    print("一致度：{0} | {1}".format(r[1], r[0]))

d_pos = [docs[i] for i in relation[q2]['pos']]
d_neg = [docs[i] for i in relation[q2]['neg']]
print("pos: {}".format(d_pos))
print("neg: {}".format(d_neg))

v_pos = [convert_doc_to_vector(x, corpus) for x in d_pos]
v_neg = [convert_doc_to_vector(x, corpus) for x in d_neg]

v_q2 = convert_doc_to_vector(q2, corpus)
q = Rocchio(v_q2, v_pos, v_neg)
print("")
print("===== Rocchioアルゴリズムを適用した状態で検索 =====")
print("クエリ：{0}".format(q2))
print("=================================")
for r in _nearest(q, docs):
    print("一致度：{0} | {1}".format(r[1], r[0]))

print("")
print("ちゃんと虚数単位 i に関するドキュメントの一致度が上がり、他のものは下がっている")
print("")


===== 何もフィードバックしない状態で検索 =====
クエリ：明日のMTGってどうなった？
一致度：0.5533985905294664 | 明日の打合せの予定はどうなっていますか？
一致度：0.2834733547569204 | 明日の会議室はとれましたか？
一致度：0.12499999999999997 | あなたの iPad は、町内会の福引の景品でもらったものですね？
一致度：0.0 | 僕の iPhone は床に叩きつけて壊れました。
一致度：0.0 | 虚数単位 i は、自乗して -1 になる数と定義したものです。

クエリに Rocchioアルゴリズムによるフィードバックを与えてみる。

pos: ['明日の会議室はとれましたか？', '明日の打合せの予定はどうなっていますか？']
neg: ['僕の iPhone は床に叩きつけて壊れました。', 'あなたの iPad は、町内会の福引の景品でもらったものですね？', '虚数単位 i は、自乗して -1 になる数と定義したものです。']

===== Rocchioアルゴリズムを適用した状態で検索 =====
クエリ：明日のMTGってどうなった？
一致度：0.7329001121935711 | 明日の打合せの予定はどうなっていますか？
一致度：0.4887459813054017 | 明日の会議室はとれましたか？
一致度：0.004973462772370439 | あなたの iPad は、町内会の福引の景品でもらったものですね？
一致度：-0.05312145630300424 | 僕の iPhone は床に叩きつけて壊れました。
一致度：-0.09474456055826497 | 虚数単位 i は、自乗して -1 になる数と定義したものです。

iPad の件の文章の一致度が下がってていい感じ
続いてもう一つのクエリについて

===== 何もフィードバックしない状態で検索 =====
クエリ：i の2乗っていくつになるか知ってる？
一致度：0.27386127875258304 | 明日の打合せの予定はどうなっていますか？
一致度：0.21997067253202998 | 虚数単位 i は、自乗して -1 になる数と定義したものです。
一致度：0.1091089451179962 

### 上記のスクリプトを用いて検索と Rocchioアルゴリズムの検証を行った

#### 対象文書

* 明日の会議室はとれましたか？
* 明日の打合せの予定はどうなっていますか？
* 僕の iPhone は床に叩きつけて壊れました。
* あなたの iPad は、町内会の福引の景品でもらったものですね？
* 虚数単位 i は、自乗して -1 になる数と定義したものです。

#### クエリ

* 明日のMTGってどうなった？
* i の2乗っていくつになるか知ってる？

#### 関連文書の定義

クエリそれぞれに対して、関連、非関連文書の定義を以下のようにした：

##### 明日のMTGってどうなった？

* 関連文書
   * 明日の会議室はとれましたか？
   * 明日の打合せの予定はどうなっていますか？

* 非関連文書
   * 僕の iPhone は床に叩きつけて壊れました。
   * あなたの iPad は、町内会の福引の景品でもらったものですね？
   * 虚数単位 i は、自乗して -1 になる数と定義したものです。

#####  i の2乗っていくつになるか知ってる？

* 関連文書
   * 虚数単位 i は、自乗して -1 になる数と定義したものです。

* 非関連文書
   * 明日の会議室はとれましたか？
   * 明日の打合せの予定はどうなっていますか？
   * 僕の iPhone は床に叩きつけて壊れました。
   * あなたの iPad は、町内会の福引の景品でもらったものですね？


### 結果

思ったようにフィードバックアルゴリズムが機能している。

```
===== 何もフィードバックしない状態で検索　=====
クエリ：明日のMTGってどうなった？
=================================
一致度：0.5533985905294664 | 明日の打合せの予定はどうなっていますか？
一致度：0.2834733547569204 | 明日の会議室はとれましたか？
一致度：0.12499999999999997 | あなたの iPad は、町内会の福引の景品でもらったものですね？
一致度：0.0 | 僕の iPhone は床に叩きつけて壊れました。
一致度：0.0 | 虚数単位 i は、自乗して -1 になる数と定義したものです。

クエリに Rocchioアルゴリズムによるフィードバックを与えてみる。

pos: ['明日の会議室はとれましたか？', '明日の打合せの予定はどうなっていますか？']
neg: ['僕の iPhone は床に叩きつけて壊れました。', 'あなたの iPad は、町内会の福引の景品でもらったものですね？', '虚数単位 i は、自乗して -1 になる数と定義したものです。']

===== Rocchioアルゴリズムを適用した状態で検索 =====
クエリ：明日のMTGってどうなった？
=================================
一致度：0.7329001121935711 | 明日の打合せの予定はどうなっていますか？
一致度：0.4887459813054017 | 明日の会議室はとれましたか？
一致度：0.004973462772370439 | あなたの iPad は、町内会の福引の景品でもらったものですね？
一致度：-0.05312145630300424 | 僕の iPhone は床に叩きつけて壊れました。
一致度：-0.09474456055826497 | 虚数単位 i は、自乗して -1 になる数と定義したものです。

iPad の件の文章の一致度が下がってていい感じ
続いてもう一つのクエリについて

===== 何もフィードバックしない状態で検索 =====
クエリ：i の2乗っていくつになるか知ってる？
=================================
一致度：0.27386127875258304 | 明日の打合せの予定はどうなっていますか？
一致度：0.21997067253202998 | 虚数単位 i は、自乗して -1 になる数と定義したものです。
一致度：0.1091089451179962 | 明日の会議室はとれましたか？
一致度：0.07216878364870323 | あなたの iPad は、町内会の福引の景品でもらったものですね？
一致度：0.0 | 僕の iPhone は床に叩きつけて壊れました。

pos: ['虚数単位 i は、自乗して -1 になる数と定義したものです。']
neg: ['明日の会議室はとれましたか？', '明日の打合せの予定はどうなっていますか？', '僕の iPhone は床に叩きつけて壊れました。', 'あなたの iPad は、町内会の福引の景品でもらったものですね？']

===== Rocchioアルゴリズムを適用した状態で検索 =====
クエリ：i の2乗っていくつになるか知ってる？
=================================
一致度：0.8073483049049393 | 虚数単位 i は、自乗して -1 になる数と定義したものです。
一致度：0.12876003579062803 | あなたの iPad は、町内会の福引の景品でもらったものですね？
一致度：0.10935556732028369 | 明日の打合せの予定はどうなっていますか？
一致度：0.03893337525780316 | 明日の会議室はとれましたか？
一致度：0.03610782578622698 | 僕の iPhone は床に叩きつけて壊れました。

ちゃんと虚数単位 i に関するドキュメントの一致度が上がり、他のものは下がっている
```


### 結論

Rocchio のアルゴリズムは、ユーザークエリに対して、そのクエリに関連する文書の集合と関連しない文書の集合を定義することで、
クエリを補正するように働くアルゴリズムである。
従って、このアルゴリズムを適用した場合、通常は関係無いと判断されるドキュメントの検索結果が上位に表示されてしまうことは有り得る
（実際に、`i の2乗っていくつになるか知ってる？` に対して `僕の iPhone は床に叩きつけて壊れました。` の一致度がやや上昇している）。

しかしながら、関連文書を正しく定義してあげれば、このアルゴリズムはフィードバックを受け付けてより UX の高い検索結果を提示する方向に
クエリを修正するのではないか、と考えられる。